### **Nurse Scheduling**
Hospital administrators must schedule nurses so that the hospital’s patients are provided adequate care. At the same time, careful attention must be paid to keeping costs down. From historical records, administrators can project the minimum number of nurses required to be on hand for various times of day and days of the week. 

The objective is to find the minimum total number of nurses required to provide adequate care.Nurses start work at the beginning of one of the four-hour shifts given below (except for shift 6) and work for 8 consecutive hours. Hence, possible start times are the start of shifts 1 through 5. Also, assume that the projected required number of nurses factors in time for each nurse to have a meal break.

Formulate and solve the nurse scheduling problem as an integer program for one day for the data given below.
Hint: Note that exceeding the minimum number of needed nurses in each shift is acceptable so long as the total number of nurses over all shifts is minimized.

<table>
    <thead>
        <tr>
            <th> Time </th>
            <th> No. of Nurses </th>
        </tr>
    </thead>
    <tbody>
      <tr>
        <td>12:00 a.m. – 4:00 a.m.</td>
        <td>10</td>
      </tr>
      <tr>
        <td>4:00 a.m. – 8:00 a.m.</td>
        <td>24</td>
      </tr>
      <tr>
        <td>8:00 a.m. – 12:00 p.m.</td>
        <td>18</td>
      </tr>
      <tr>
        <td>12:00 p.m. – 4:00 p.m.</td>
        <td>10</td>
      </tr>
      <tr>
        <td>4:00 p.m. – 8:00 p.m.</td>
        <td>23</td>
      </tr>
      <tr>
        <td>8:00 p.m. – 12:00 a.m.</td>
        <td>17</td>
      </tr>
    </tbody>
</table>

## **Sets**
$\mathcal{T}$: Set of shifts.

$\mathcal{S}$: Set of nurse start times.

## **Indecies**
$t \in \mathcal{T}$: index of shift in shifts.

$s \in \mathcal{S}$ index of start time in possible nurse start times.

## **Data**
$n_{t}, t \in \mathcal{T}$, Amount of needed nurses at each time step.

$w_{st}, t \in \mathcal{T}, s \in \mathcal{S}$ binary value indicating whether the nurses who started at shift $s$ is working during shift $t$.
## **Decision Variables**
$x_s, s \in \mathcal{S}$ Number of nurses starting at time $s$.
## **Formulation**
**Objective function**
\begin{align*}
\mathrm{Min} \sum_{s \in \mathcal{S}} x_{s}
\end{align*}

**S.T.**
\begin{gather}
\sum_{s \in \mathcal{S}}x_{s} w_{st} \ge n_{t}, \forall t \in \mathcal{T} \\
x_{s} \ge 0, \forall s \in \mathcal{S} \\
x_{s} \in \mathbb{Z}, \forall s \in \mathcal{S}\\
\end{gather}


In [1]:
from docplex.mp.model import Model
model = Model(name = "Nurse Scheduling")

In [2]:
# Data
T = list(range(6))
S = list(range(5))

n = [10, 24, 28, 10, 23, 17]
w = [
    [1, 1, 0, 0, 0, 0],
    [0, 1, 1, 0, 0, 0],
    [0, 0, 1, 1, 0, 0],
    [0, 0, 0, 1, 1, 0],
    [0, 0, 0, 0, 1, 1]
]


In [3]:
x = model.integer_var_list(len(S), name = "x", lb = 0)

In [4]:
model.minimize(model.sum(x[s] for s in S))

In [5]:
for t in T:
    model.add_constraint(model.sum(w[s][t] * x[s] for s in S) >= n[t])

In [6]:
model.export_as_lp("Nurse_Scheduling.lp")

'Nurse_Scheduling.lp'

In [7]:
model.solve(log_output = True)

Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
Found incumbent of value 102.000000 after 0.02 sec. (0.00 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 2 rows and 0 columns.
Reduced MIP has 4 rows, 5 columns, and 8 nonzeros.
Reduced MIP has 0 binaries, 5 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (0.00 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 4 rows, 5 columns, and 8 nonzeros.
Reduced MIP has 0 binaries, 5 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 16 threads.
Root relaxation solution time = 0.00 sec. (0.00 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

*     0+    0                          102.0000       27.0000  

docplex.mp.solution.SolveSolution(obj=61,values={x_0:10,x_1:14,x_2:14,x_..

In [10]:
obj = model.objective_value
assignment = [x[s].solution_value for s in S]

for t in T:
    print(f"Day {t + 1}, will have {sum(w[s][t] * assignment[s] for s in S)}")
    assert sum(w[s][t] * assignment[s] for s in S) >= n[t]

Day 1, will have 10.0
Day 2, will have 24.0
Day 3, will have 28.0
Day 4, will have 14.0
Day 5, will have 23.0
Day 6, will have 23.0
